- In this Script we have converted the R script of text analytics to Python Script.
We are tagging each SR and complaints with the possible root cause tag based on the SN remarks column 

In [12]:
# Import libraries for 
import pandas as pd
import numpy as np
import sys
sn_remarks = pd.read_csv("C:\\Users\\ijohan\\Desktop\\ai_project\\source_data\\july\\20190723July_SRC_Listing_latest.csv")

In [107]:
#sn_remarks.columns

In [13]:
sn_remarks.columns

Index(['State', 'State Description', 'Business Area - Key', 'Channel',
       'Complaint ID', 'CA Number', 'Unit', 'Category', 'Sub-Category',
       'Creation Date', 'SN Number', 'SO Number', 'BPEM Case', 'Notes',
       'SN_Remark', 'BPEM Notes 29July', 'Feedback Status', 'Status 29July',
       'Verified Root Cause Type', 'Verified Root Cause', 'Root Cause Measure',
       'Refund/Arrears', 'Dt/Cr Amount', 'MB Reason'],
      dtype='object')

- data preprocessing on sn remarks column

In [14]:
#https://stackoverflow.com/questions/8270092/remove-all-whitespace-in-a-string-in-python
from nltk.corpus import stopwords
import re
import string
  
def clean_txt(text):
    text = str(text).upper()
    text = text.replace("PIHAK TUAN PUAN", " ").replace("NBSP NBSP NBSP", " ").replace(" KERJASAMA PIHAK TUAN", " ").replace("BILLING", "BIL") \
                            .replace("TINGG I", "TINGGI").replace("GMAIL", " ").replace("INTERNAL NOTE", "") \
                           .replace(". .", "").replace("PERMOHONAN SEMAK METER NO TELEFON", " ").replace("PENGGUNA MAKLUM", " ")\
                           .replace("TELAH HUBUNGI PENGGUNA DAN", " ").replace("BA AAN", "BACAAN").replace("TUAN PUAN", " ")
#                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
#                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
#                          .replace("€", " euro ").replace("'ll", " will")
    text = re.sub(r'\[[0-9]*\]',' ',str(text))
    text = re.sub(r'\s+',' ',str(text))
    text = re.sub(r'\W',' ',text) #remove non word character
    text = re.sub(r'[@#$%\*\(\)\<\>\?\'\":;\]\[-]/',' ',str(text))
    text = re.sub(r'\d',' ',str(text))
    text = re.sub(r'\s+',' ',str(text))
    text  = re.sub("^\s+|\s+$", "",str(text))
   
    return text

In [15]:
sn_remarks['clean_sn_remark']  = sn_remarks['SN_Remark'].apply(clean_txt)

In [22]:
sn_remarks[['clean_sn_remark','SN_Remark']].head()

,clean_sn_remark,SN_Remark
0,PENGGUNA AMKLUM SEMASA BELIAU MEMBUAT RENOVATI...,Internal note 03.07.2019 10:48:27 60068244 -...
1,TUKAR METER BARU METER BAKAR HANCUR LEBURKONTR...,Internal note 11.07.2019 11:56:29 10075024 I...
2,PENGGUNA MOHON UNTUK TUKAR METER PENGGUNA TELA...,7192005851Internal note 10.07.2019 11:12:52 1...
3,METER TIADA PAPARAN BEKALAN ELEKTRIK MASIH ADA...,Internal note 08.07.2019 15:49:44 60068195 M...
4,MERUJUK KEPADA PERKARA DI ATAS PENGGUNA TELAH ...,Internal note 15.07.2019 19:44:18 ...


In [23]:
# Below function create ngrames of the documents, as a arguments it takes text and n. text: text document, n = number of grams
# ex: n= 2, gives 2 gram
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
def get_ngrams(text, n):
    #n = 3
    n_grams = ngrams(word_tokenize(text), n)
    return [ ' '.join(grams) for grams in n_grams]

# Applying above function on the SN_remark column
sn_remarks['clean_sn_remark_grams1']  = sn_remarks['clean_sn_remark'].astype(str).apply(get_ngrams, n=1)
sn_remarks['clean_sn_remark_grams2']  = sn_remarks['clean_sn_remark'].astype(str).apply(get_ngrams, n=2)
sn_remarks['clean_sn_remark_grams3']  = sn_remarks['clean_sn_remark'].astype(str).apply(get_ngrams, n=3)
#sn_remarks['clean_sn_remark_grams4']  = sn_remarks['clean_sn_remark'].astype(str).apply(get_ngrams, n=4)


In [24]:
# Combine all the n grams into one column
sn_remarks['clean_sn_remark_grams'] = sn_remarks['clean_sn_remark_grams1'] + sn_remarks['clean_sn_remark_grams2'] +sn_remarks['clean_sn_remark_grams3']# + sn_remarks['clean_sn_remark_grams4']

In [25]:
del sn_remarks['clean_sn_remark_grams1']
del sn_remarks['clean_sn_remark_grams2']
del sn_remarks['clean_sn_remark_grams3']

In [26]:
sn_remarks.head()

,State,State Description,Business Area - Key,Channel,Complaint ID,CA Number,Unit,Category,Sub-Category,Creation Date,...,Feedback Status,Status 29July,Verified Root Cause Type,Verified Root Cause,Root Cause Measure,Refund/Arrears,Dt/Cr Amount,MB Reason,clean_sn_remark,clean_sn_remark_grams
0,KUL,WP KUALA LUMPUR,6121,Careline,2000125485,2.200000e+11,Device Operation (OPC),Meter Faulty,Meter Broken,7/3/2019,...,Resolved,Verified,A - Meter Faulty,Meter Replaced,MREPLACE AND BACKBILL,Refund,-8.55,PELARASANDISEBABKANBACAANSALAH,PENGGUNA AMKLUM SEMASA BELIAU MEMBUAT RENOVATI...,"[PENGGUNA, AMKLUM, SEMASA, BELIAU, MEMBUAT, RE..."
1,KUL,WP KUALA LUMPUR,6121,Kedai Tenaga,2000126751,2.200000e+11,Device Operation (OPC),Meter Faulty,Meter Burnt,7/11/2019,...,Resolved,Resolved,A - Meter Faulty,METER IRREGULARITIES,MREPLACE AND BACKBILL,Refund,-563.63,METERROSAK,TUKAR METER BARU METER BAKAR HANCUR LEBURKONTR...,"[TUKAR, METER, BARU, METER, BAKAR, HANCUR, LEB..."
2,SEL,SELANGOR,6140,Kedai Tenaga,2000126546,2.200000e+11,Device Operation (OPC),Meter Faulty,Meter Blank,7/10/2019,...,Resolved,Resolved,A - Meter Faulty,METER IRREGULARITIES,MREPLACE AND BACKBILL,Refund,-90.56,KKB-METERROSAK,PENGGUNA MOHON UNTUK TUKAR METER PENGGUNA TELA...,"[PENGGUNA, MOHON, UNTUK, TUKAR, METER, PENGGUN..."
3,SEL,SELANGOR,6153,Careline,2000126327,2.200000e+11,Device Operation (OPC),Meter Faulty,Meter Blank,7/8/2019,...,Resolved,Verified,A - Meter Faulty,METER IRREGULARITIES,MREPLACE AND ADJ REV,Refund,-31.47,NaN,METER TIADA PAPARAN BEKALAN ELEKTRIK MASIH ADA...,"[METER, TIADA, PAPARAN, BEKALAN, ELEKTRIK, MAS..."
4,JOH,JOHOR,6345,Kedai Tenaga,2000127335,2.400000e+11,Device Operation (OPC),Meter Faulty,Meter Blank,7/15/2019,...,Resolved,Resolved,A - Meter Faulty,METER IRREGULARITIES,MREPLACE AND FULL REV,Refund,-3702.53,NaN,MERUJUK KEPADA PERKARA DI ATAS PENGGUNA TELAH ...,"[MERUJUK, KEPADA, PERKARA, DI, ATAS, PENGGUNA,..."


In [27]:
# assigning keywords for customer_consumption
customer_consumption_keywords = ['BERPUASHATI','ADALAH TERATUR','WAIVE ILP','PELARASAN BLOCK','METER PEMBILAN OK','TIADA MASALAH PEMBILAN','DEPAT MENERIMA','KIRAN TEPAT','TEPAT','BETUL',
                                 'METER BETUL','SEMATAN BACCAN METER','TIADA KEROSAKAN','PENERANGAN','BERSETUJU','TUKAR ALAMAT',
                                 'KAMPUNG','TAMAN','STICKER KOYAK','MASALAH PENGGUNAAN','BACAAN ADALAH TEPAT',
                                 'BACAAN ADALAH BENAR','BACAAN ADALH BENAR','BACAA N ADALAH BENAR','BACAAN ADALAH BETUL',
                                 'PEMBACA METER BETUL','BACAAN TERATUR','BETUL DAN TEPAT','ADALAH TEPAT','BACAAN TEPAT',
                                 'ACAAN BETUL','BACAAN METER BETUL','BAC AAN ADALAH BETUL','ADALAH BETUL','READING TEPAT',
                                 'BA CAAN TERATUR','BENAR DAN TEPAT','BCN ADALAH BETU','ADALAH BENAR','BACAAN YG BETUL',
                                 'BACAAN BIL BULANAN BETUL','DIDAPATI BETUL','BACAAN YANG BENAR','BACAAN NORMAL',
                                 '4 BUAH RUMAH GUNA SATU METER','TIADA MA SALAH PADA BACAAN','BACAAN DALAM KEADAAN TERATUR',
                                 'TIADA MASALAH DALAM BACAAN','ADALAH BE TUL DAN TEPAT','TAMBAHAN 1 AIR COND',
                                 'BACAAN TIADA MASALAH','DAPATI BACA AN OK','TIADA MASALAH PADA BA CAAN METER',
                                 'TIADA MASALAH PADA BACAAN METER','RDG OK','TIADA MASAALAH BACAAN',
                                 'TIADA MASALAH PADA BACAAN ME TER','TIADA MASALAH PADA BACAAN','TIADA MASALAH PAD A BACAAN',
                                 'PASANG COWAY','PENGGUNAAN ELEKTRI K TINGGI','ADALAH BETU','SEMAKAN BACAAN ADALAH BE TUL',
                                 'TIADA MASALA H DGN BACAAN','BACAAN A DALAH BETUL','BACAAN AFALAH BETUL DAN TERATUR',
                                 'BILLING TERATUR','TIADA MASALAH BACAAN','TIADA MASALAH DALAM BAC','BACAAN YG BENAR',
                                 'BACAAN METER ADALA BETUL','TIADA KESALAHAN BACAAN','BACAAN OLEH PEMBACA JAN GKA BETUL',
                                 'BACAAN YANG SEBENAR DAN TEPAT','BACAAN AD ALAH BETUL','BACAAN OK','BACAAN ADA LAH BETU',
                                 'PENGGUNAAN 22AMP3ECON','BACAAN DI METER BETUL','BACAAN YG DIAMBIL PEMBACA METER BETUL',
                                 'DAPATI BACCAN BETUL','BACAAN DIPERMIS ADA BETUL','BACAAN YG DIBUAT BETUL',
                                 'BACAAN ADALAH TERATUR','RDG BETUL','GUNAKAN PERKAKAS ELEKTRIK IAITU OVEN','GUNA PENGHAWA DINGIN 2HP PADA SIANG HARI','ADA 4 A-CON','31 HARI','31HARI','CHECK METER NORMAL','BILANGAN HARI >31HARI YANG MENYEBABKAN BIL TINGGI','KWH ADALAH BETUL','BACAAN ADALAH BETUL DAN TEPAT','BETUL TIADA MASALAH ','TIADA KESALAHAN PADA BACAAN','BERSETUJU DENGAN PENJELASAN','TIADA PEMBETULAN BIL DI BUAT','SUDAH DISELARASKAN','OKRALAT','AIKRALAT','KEADAAN BAIK','JANGKA BAIK','BAIK','METER OK','GOOD CONDITION','NORMALRALAT','TIDAK ROSAK','JANGKA OK','MTR OK','DISAHKAN TIADA MASAALAH','DALAM RALAT YANG DIBENARKAN','TIADA KEJANGGALAN METER','RALAT % ERROR TIDAK MELEBIHI JULAT','BAIKERROR','KEJITUAN','METER DIUJI BAIK','CHECK METER NORMAL']

In [28]:
# create column named keywords and fill it with respective keys words in all the rows of the column
sn_remarks['keywords'] = [customer_consumption_keywords for _ in range(len(sn_remarks))]


In [29]:
# This fucntion searches for each keywords of 'keywords' column in to 'clean_sn_reamrk' column and if matches found returns 1 else 0
# Define function to search for words in two columns
def search_func(row):
    matches = [test_value in row["clean_sn_remark_grams"]
               for test_value in row["keywords"]]

    if any(matches):
        return 1
    else:
        return 0




In [30]:
# Apply the matching fucntion on sn remarks column
sn_remarks['customer_consumption_keywords_flag'] = sn_remarks.apply(search_func, axis=1)

In [31]:
# count number of matches based on the number of 1 and 0 in the respective columns
sn_remarks.customer_consumption_keywords_flag.value_counts()

0    9372
1    8650
Name: customer_consumption_keywords_flag, dtype: int64

In [32]:
## assigning keywords for meter_quality_keywords
meter_quality_keywords = ['DI POTONG','STOLEN','BURNING','KACA YANG PECAH', 'METER BURNT','METER TELAH PECAH','BERHENTI',
                          'METER LOST','MELETOP','METER BAKAR','TIDAK KELUARKAN PAPARAN','METER BERGERAK PERLAHAN', 'PECAH',
                          'METER KENA KILAT', 'PECAH PETIR', 'KENE PETIR','METER PAPAR ERROR','METER   FASA ROSAK', 'FASA BAKAR',
                          'JANGKA TERBAKAR','FASA SHORT','METER HENTI','METER STOPTUKAR','TERBAKAR','DI POTONG','METER AGEINGT',
                          'METER SHORTING','TUKAR MTR','METER RALAT','METER MEKANIKAL','NO PAPARAN',
                          'METER USANG','TIDAK MENYALA','BEKALAN MASIH ADA','METER TERHENTI','CAJ MINIMA','METER SAMAR',
                          'METER BLANK','METER BROKEN','METER PICAH','GANTI METER','TIDAK BERGERAK','METER HILANG',
                          'METER PREMISE','SYAK','METER ROSAK','METER ROSK','METER KOTOR','MTR USANG','METER KABUR',
                          'JANGKA USANG','OLD METER','CHANGE METER','METER BERGERAK LAJU','JANGKA PENGGUNA LAJU',
                          'TIDAK MENEPATI','METER SILAU','DIDAPATI ROSAK','ROSK','AGED METER','TUKAR METER BARU','ACCURACY METER OUT','METER MELEBIHI 10 TAHUN','MTR ROSAK','METER LAMA','METER SLOW','+5','+4','-5','-4','METER LEBIH 10 TAHUN','METER LOMPAT','JANGKA KABUR DAN USANG','KEJANGGALAN METER AMAT TINGGI','PIRING JANGKA TAK STABIL','RALAT METER TIDAK MENGIKUT','SUDAH TUKAR JANGKA BARU','ERROR TINGGI','METER LAJU','METER BLANK/BURNT','READING STOP','ROSAK-BLANK','GERAK LAJU','JANGKA LAJU','JANGKA BARU','USANG','AGED','TERHENTI','METER PECAH','-8','METER TIADA PAPARAN','METER BERHENTI','ERROR 32.98%','TIADA PAPARAN','RDG STOP','METER TERBAKAR','METER DAN PREMIS TERBAKAR','DISPLAY ROSAK','JANGKA ROSAK','BLANK','METER PERLAHAN"','KEMBUNG','METER YG TERBAKAR','PENGGUNA TETAP TIDAK BERPUAS HATI','DAIL','BACAAN ERROR','PAPARAN METER TERHALANG','METER PUDAR','TIDAK BERFUNGSI','DISAMBAR PETIR','CHOYKWHMETER ROSAK','SEMAK METER ROSAK','METER ROSAK X','METER ROSAKRUMAH TIADA','METER ROSAK MINTAHUBUNGI','METER ROSAK STOPCUST','LAB TEAMMETER ROSAKPERLU','METER ROSAKSEMAKN DAPATI','PERUKARAN METER ROSAKTIADA','METER ROSAKTERHENTI ','METER ROSAK TQ','METER ROSAK PERLU','AMIRUDDINMETER ROSAK','METER ROSAKBUAT','METER ROSAKSEMAKN','METER ROSAK PENGGUNAAN','BETUL RMETER ROSAKM','AI LEEMETER ROSAKMOHON','DIRUMAHTEMUJANJI ESOKMETER ROSAKTUKAR','SALAHATAU METER ROSAK','DISYAKI METER ROSAKRUMAH','JULIANA METER ROSAK','TEAMMETER ROSAKPERLU','KWHMETER ROSAKTRANSFER','FAKIRRADZILR KWHMETER ROSAKAVERAGE','KUATIR METER ROSAK','METER ROSAK ','METER ROSAKPENGGUNA BERITAHU','ENMUNIADY METER ROSAK','METER ROSAK SERTA','APPOINTMENT METER ROSAK','DISYAKI METER ROSAKADA','METER ROSAKTRANSFER KE','RISAU METER ROSAK','METER ROSAKSEMAKA TELAH','DIKHUATIRI METER ROSAK','NURUL METER ROSAK','LOONG METER ROSAK','METER ROSAKADA','METER ROSAKBUAT TUKAR','SUSPECT METER ROSAKNO','TERDAPAT METER ROSAK','METERR METER ROSAKM','METER ROSAKBACAAN TIDAK','METER ROSAKM INTERNAL','METER ROSAKRUMAH','METER ROSAKDAN CUST','TET CHOYKWHMETER ROSAK','PERLAHAN METER ROSAK','METER ROSAK PEGGUNA','AMIRUDDINMETER ROSAK LAST','MENDAPATI METER ROSAKGANTI','FAULFINDER METER ROSAK','BBRQ METER ROSAK','SIVAM RMETER ROSAKM','SAIDRMMETER ROSAK','MR VISHALMETER ROSAK','METER ROSAK NUMBER','MIZAMETER ROSAK CEPER','TERHENTIBIL METER ROSAK','ROSAKMETER ROSAKTUKAR SERTA','METER ROSAK MISS','JANGKA METER ROSAKPURATA','DITAPAKMETER ROSAK ERROR','SYAKI METER ROSAKBIL','METER ROSAK BULA','METER ROSAKERROR','DAN METER ROSAK','METER ROSAK TERKENA','TERPASANGDISYAKI METER ROSAKMOHON','METER ROSAK ATAUSEBAGAINYAADUAN','TERTULIS METER ROSAK','MOHDMETER ROSAKTIADA','METER ROSAK ASSIGN','METER ROSAKLAJU INTERNAL']

In [33]:
sn_remarks['keywords'] = [meter_quality_keywords for _ in range(len(sn_remarks))]

In [34]:
# Define function to search for words in two columns
def search_func(row):
    matches = [test_value in row["clean_sn_remark_grams"]
               for test_value in row["keywords"]]

    if any(matches):
        return 1
    else:
        return 0




In [35]:
sn_remarks['meter_quality_keywords_flag'] = sn_remarks.apply(search_func, axis=1)

In [36]:
sn_remarks.meter_quality_keywords_flag.value_counts()

0    16373
1     1649
Name: meter_quality_keywords_flag, dtype: int64

In [37]:
meter_crossing_keywords = ['SALAH PASANG','TERSALAH DIPASANG','JANGKA HILANG','METER CROSSING','SILANG','CROSSING','CROSING','BERSILANG']

In [38]:
# Add new key word column for each column
sn_remarks['keywords'] = [meter_crossing_keywords for _ in range(len(sn_remarks))]

In [39]:
def search_func(row):
    matches = [test_value in row["clean_sn_remark_grams"]
               for test_value in row["keywords"]]

    if any(matches):
        return 1
    else:
        return 0

In [40]:
sn_remarks['meter_crossing_keywords_flag'] = sn_remarks.apply(search_func, axis=1)

In [41]:
sn_remarks.meter_crossing_keywords_flag.value_counts()

0    17958
1       64
Name: meter_crossing_keywords_flag, dtype: int64

In [42]:
## assigning keywords for incorrect_reading_keywords
incorrect_reading_keywords = ['TERKURANG CAJ','TERSALAH MASUK BACAAN KVARH','SALAHBACAAN','BIL SALAH','SALAH BACA','TERSALAH READING','TIDAK TEPAT','BACAN SALAH','TIDAKTEPAT','WRONG KEY IN','TERSALAH AMBIL BACAAN','BACA AN SALAH','BACA SALAH','PEMBETULAN BACAAN','PEMBETULAN BA CAAN','BACAAN SALAH','RALAT BACAAN','BUAT BILL BARU','BACAAN SALA','KESILAPAN BACAAN','SALAH BACAAN','BETULKAN BACAAN','BACAAN SA LAH','BACAAN ADALAH SALAH','SALAH RDG','METEREDER SALAH','OVEREADING','SALAH MEMBUAT BACAAN','OVERREAD','BACAAN SUDAH DIBETULKAN','TERSALAH MENGAMBIL BACAAN','DIBETULKAN DENGAN BACAAN SEBENAR','BACCAN SUDAH DIBETULKAN','SALAH MMBUAT BACAAN','WRONG READING','BACAAN KVARH SALAH','BILL TELAH DIBETULKAN MENGIKUT BACAAN','OVER READ','ADALAH SALAH','SALAH READING','RDG OVERRE AD','PEMBACA JANGKA TERSALAH','BACAAN METER SALAH','MR TERKURANG BACA','BACAAN MR BULAN 3 RENDAH','OVEREAD READING','TERSALAH BUAT BACAAN','KESALAHAN BACAAN','BACAAN UNDER READ','SALAH TECO READING','BIL TERSILAP BACA','BACAAN ADALAH KOSONG','BACAAN APRIL RENDAH','BACAAN DIBACA RENDAH','TERSALAH BACA','SALAH BACCAN','SILAP BACA','TERSALAH MASUKKAN BACAAN','BACAAN DAHULU SALAH','PEMBACA JANGKA BACA SALAH','YG TIDAK TEPAT','TIDAK TEPATPENGUNA','ADALAH TIDAK TEPATPENGUNA','TIDAKTEPATPENGGUNA MEM','TIDAK TEPATMOHON','ADALAH TIDAKTEPATPENGGUNA','TIDAK TEPAT PENJELASAN','TIDAK TEPATPENGGUNA','TIDAK TEPATPEMB ETULAN','BACAAN TIDAK BOLEHDIBETULKAN','BACAAN PEMBETULAN TIDAK','BACAAN BETULTIDAK','TIDAK BERAPA BETULPERGERAKAN','BACAAN BETULTIDAK DAPAT','UNDER READ']

In [43]:
# Add new key word column for each column
sn_remarks['keywords'] = [incorrect_reading_keywords for _ in range(len(sn_remarks))]

In [44]:
def search_func(row):
    matches = [test_value in row["clean_sn_remark_grams"]
               for test_value in row["keywords"]]

    if any(matches):
        return 1
    else:
        return 0

In [45]:
sn_remarks['incorrect_reading_keywords_flag'] = sn_remarks.apply(search_func, axis=1)

In [46]:
sn_remarks.incorrect_reading_keywords_flag.value_counts()

0    16907
1     1115
Name: incorrect_reading_keywords_flag, dtype: int64

In [47]:
high_bill_keywords = ['2 KALI GANDA','KALI GANDA','GANDA']

In [48]:
# Add new key word column for each column
sn_remarks['keywords'] = [high_bill_keywords for _ in range(len(sn_remarks))]

In [49]:
sn_remarks['high_bill_keywords_flag'] = sn_remarks.apply(search_func, axis=1)

In [50]:
sn_remarks.high_bill_keywords_flag.value_counts()

0    17878
1      144
Name: high_bill_keywords_flag, dtype: int64

In [51]:
long_billing_period_keywords = ['PRORATE','MELEBINI 30 HARI','LAMBAT','LAMBAT BACA','32 HARI','33 HARI','34 HARI','35 HARI','36 HARI','LEBIH 30 HARI','62 HARI','BIL DIBACA LAMBAT','PRORATA']

In [52]:
# Add new key word column for each column
sn_remarks['keywords'] = [long_billing_period_keywords for _ in range(len(sn_remarks))]

In [53]:
sn_remarks['long_billing_period_keywords_flag'] = sn_remarks.apply(search_func, axis=1)

In [54]:
sn_remarks.long_billing_period_keywords_flag.value_counts()

0    17771
1      251
Name: long_billing_period_keywords_flag, dtype: int64

In [55]:
meter_healthy_keywords = ['JANGKA BAIKRALAT','JANGKA METER OKBERADA','JANGKA METER BERKEADAAN BAIKBERADA','JANGKA METER BAIKBERADA','TESTMETER OK','RDG TIADA MASALAH','OKRALAT','AIKRALAT','KEADAAN BAIK','JANGKA BAIK','BAIK','METER OK','GOOD CONDITION','NORMALRALAT','TIDAK ROSAK','JANGKA OK','MTR OK','DISAHKAN TIADA MASAALAH','DALAM RALAT YANG DIBENARKAN','TIADA KEJANGGALAN METER','RALAT % ERROR TIDAK MELEBIHI JULAT','BAIKERROR','KEJITUAN','METER DIUJI BAIK','CHECK METER NORMAL']

In [56]:
# Add new key word column for each column
sn_remarks['keywords'] = [meter_healthy_keywords for _ in range(len(sn_remarks))]

In [57]:
sn_remarks['meter_healthy_keywords_flag'] = sn_remarks.apply(search_func, axis=1)

In [58]:
sn_remarks.meter_healthy_keywords_flag.value_counts()

0    13656
1     4366
Name: meter_healthy_keywords_flag, dtype: int64

In [59]:
smart_meter_keywords = ['SMARTMETER','SMART','SMART METER','BILIK GERAKAN SM','UNIT SM','SM','MENGGUNAKANSMART METER','SMOC','SELEPAS TUKAR SMART METER']

In [60]:
# Add new key word column for each column
sn_remarks['keywords'] = [smart_meter_keywords for _ in range(len(sn_remarks))]

In [61]:
sn_remarks['smart_meter_keywords_flag'] = sn_remarks.apply(search_func, axis=1)

In [62]:
sn_remarks.smart_meter_keywords_flag.value_counts()

0    17507
1      515
Name: smart_meter_keywords_flag, dtype: int64

In [63]:
estimate_bill_keywords = ['BACAANANGGARAN', 'EAT BILL', 'EST BILL','ANGGARANKERANA','METER TERHALANG','TTDI BILLING ISSUE','TTDI HIGH BILL','TUN DR ISMAIL','HIGH BILL ISSUE','TMN TUN DR','BCAAN ANGARAN','BIL ANG GARAN','BACAAN EST','BILL N','BACAAN AGGARAN','BACAAN SEBENAR','E BILL','BIL E','BIL ANGGARAN','ANGGARAN','BACAAN ESTIMATE','ESTIMATE','SYSTEM EST RENDAH','BIL ANGGA RAN','BIL ELEKTRIK ESTI MATE','BIL ELEKTRIK ESTIMA TE']

In [64]:
# Add new key word column for each column
sn_remarks['keywords'] = [estimate_bill_keywords for _ in range(len(sn_remarks))]

In [65]:
sn_remarks['estimate_bill_keywords_flag'] = sn_remarks.apply(search_func, axis=1)

In [66]:
sn_remarks.estimate_bill_keywords_flag.value_counts()

0    16521
1     1501
Name: estimate_bill_keywords_flag, dtype: int64

In [67]:
meter_tampering_keywords = ['SEAL']

In [68]:
# Add new key word column for each column
sn_remarks['keywords'] = [meter_tampering_keywords for _ in range(len(sn_remarks))]

In [69]:
sn_remarks['meter_tampering_keywords_flag'] = sn_remarks.apply(search_func, axis=1)

In [70]:
sn_remarks.meter_tampering_keywords_flag.value_counts()

0    17985
1       37
Name: meter_tampering_keywords_flag, dtype: int64

In [75]:
bill_not_received = ['TIDAK MENERIMA BIL','BIL XDAPAT']

In [76]:
# Add new key word column for each column
sn_remarks['keywords'] = [bill_not_received for _ in range(len(sn_remarks))]

In [77]:
sn_remarks['bill_not_received_flag'] = sn_remarks.apply(search_func, axis=1)

In [78]:
sn_remarks.bill_not_received_flag.value_counts()

0    18004
1       18
Name: bill_not_received_flag, dtype: int64

In [80]:
billing_issue = ['DUA BULAN CHARGESMOHON','DOUBLE CHARGE','BIL TERTUNGGAK']

In [81]:
# Add new key word column for each column
sn_remarks['keywords'] = [billing_issue for _ in range(len(sn_remarks))]

In [82]:
sn_remarks['billing_issue_flag'] = sn_remarks.apply(search_func, axis=1)

In [83]:
sn_remarks.billing_issue_flag.value_counts()

0    18022
Name: billing_issue_flag, dtype: int64

In [84]:
#sn_remarks.columns

In [85]:
del sn_remarks['clean_sn_remark_grams']
del sn_remarks['keywords'] 

In [86]:
sn_remarks['flag_sum'] = sn_remarks.iloc[:, -12:].sum(axis=1)

In [181]:
#sn_remarks.to_csv("C:\\MPS\\projects\\TNB\\billing\\test1.csv")

# Post tagging work- creating column for tags

In below section of code, we are craeting a tag column for each SR and complaint id and fill with possible RCA tag 

In [87]:
sn_remarks.columns

Index(['State', 'State Description', 'Business Area - Key', 'Channel',
       'Complaint ID', 'CA Number', 'Unit', 'Category', 'Sub-Category',
       'Creation Date', 'SN Number', 'SO Number', 'BPEM Case', 'Notes',
       'SN_Remark', 'BPEM Notes 29July', 'Feedback Status', 'Status 29July',
       'Verified Root Cause Type', 'Verified Root Cause', 'Root Cause Measure',
       'Refund/Arrears', 'Dt/Cr Amount', 'MB Reason', 'clean_sn_remark',
       'customer_consumption_keywords_flag', 'meter_quality_keywords_flag',
       'meter_crossing_keywords_flag', 'incorrect_reading_keywords_flag',
       'high_bill_keywords_flag', 'long_billing_period_keywords_flag',
       'meter_healthy_keywords_flag', 'smart_meter_keywords_flag',
       'estimate_bill_keywords_flag', 'meter_tampering_keywords_flag',
       'bill_not_received_flag', 'billing_issue_flag', 'flag_sum'],
      dtype='object')

In [88]:
cols = ['customer_consumption_keywords_flag', 'meter_quality_keywords_flag',
       'meter_crossing_keywords_flag', 'incorrect_reading_keywords_flag',
       'high_bill_keywords_flag', 'long_billing_period_keywords_flag',
       'meter_healthy_keywords_flag', 'smart_meter_keywords_flag',
       'estimate_bill_keywords_flag', 'meter_tampering_keywords_flag','bill_not_received_flag', 'billing_issue_flag',
       'flag_sum']
sn_remarks[cols] = sn_remarks[cols].replace({0.:0, 1.:1})

In [89]:
sn_remarks['possible_rca'] = sn_remarks[list(['customer_consumption_keywords_flag', 'meter_quality_keywords_flag',
       'meter_crossing_keywords_flag', 'incorrect_reading_keywords_flag',
       'high_bill_keywords_flag', 'long_billing_period_keywords_flag',
       'meter_healthy_keywords_flag', 'smart_meter_keywords_flag',
       'estimate_bill_keywords_flag','meter_tampering_keywords_flag','bill_not_received_flag', 'billing_issue_flag'])].eq(1, axis=0).apply(lambda x: ' '.join(x[x].index), axis=1)

In [91]:
sn_remarks = sn_remarks[['State', 'State Description', 'Business Area - Key', 'Channel',
       'Complaint ID', 'CA Number', 'Unit', 'Category', 'Sub-Category',
       'Creation Date', 'SN Number', 'SO Number', 'BPEM Case', 'Notes',
       'SN_Remark', 'BPEM Notes 29July', 'Feedback Status', 'Status 29July',
       'Verified Root Cause Type', 'Verified Root Cause', 'Root Cause Measure',
       'Refund/Arrears', 'Dt/Cr Amount', 'MB Reason', 'clean_sn_remark',
       'customer_consumption_keywords_flag', 'meter_quality_keywords_flag',
       'meter_crossing_keywords_flag', 'incorrect_reading_keywords_flag',
       'high_bill_keywords_flag', 'long_billing_period_keywords_flag',
       'meter_healthy_keywords_flag', 'smart_meter_keywords_flag',
       'estimate_bill_keywords_flag', 'meter_tampering_keywords_flag',
       'bill_not_received_flag', 'billing_issue_flag', 'flag_sum', 'possible_rca']]

In [92]:
#coppy of sn remarks
sn_remarks_copy = sn_remarks

In [93]:
sn_remarks.to_csv("C:\\Users\\ijohan\\Desktop\\ai_project\\source_data\\july\\JUNE_FURTHER_INVESTIGATION_run22July_rca.csv")

In [90]:
#n_remarks_copy.head()

# Adding new column for combined tags

In [94]:
sn_remarks.columns

Index(['State', 'State Description', 'Business Area - Key', 'Channel',
       'Complaint ID', 'CA Number', 'Unit', 'Category', 'Sub-Category',
       'Creation Date', 'SN Number', 'SO Number', 'BPEM Case', 'Notes',
       'SN_Remark', 'BPEM Notes 29July', 'Feedback Status', 'Status 29July',
       'Verified Root Cause Type', 'Verified Root Cause', 'Root Cause Measure',
       'Refund/Arrears', 'Dt/Cr Amount', 'MB Reason', 'clean_sn_remark',
       'customer_consumption_keywords_flag', 'meter_quality_keywords_flag',
       'meter_crossing_keywords_flag', 'incorrect_reading_keywords_flag',
       'high_bill_keywords_flag', 'long_billing_period_keywords_flag',
       'meter_healthy_keywords_flag', 'smart_meter_keywords_flag',
       'estimate_bill_keywords_flag', 'meter_tampering_keywords_flag',
       'bill_not_received_flag', 'billing_issue_flag', 'flag_sum',
       'possible_rca'],
      dtype='object')

In [93]:
#sn_remarks.head()

In [95]:
sn_remarks.loc[(sn_remarks['possible_rca'].str.contains('meter_quality_keywords_flag') & sn_remarks['possible_rca'].str.contains('estimate_bill_keywords_flag')), 'combined_possible_rca'] = 'meter_quality_keywords_flag'

In [96]:
sn_remarks.loc[(sn_remarks['possible_rca'].str.contains('meter_quality_keywords_flag') & sn_remarks['possible_rca'].str.contains('incorrect_reading_keywords_flag')), 'combined_possible_rca'] = 'meter_quality_keywords_flag'

In [97]:
sn_remarks.loc[(sn_remarks['possible_rca'].str.contains('incorrect_reading_keywords_flag') & sn_remarks['possible_rca'].str.contains('estimate_bill_keywords_flag')), 'combined_possible_rca'] = 'estimate_bill_keywords_flag'

In [98]:
sn_remarks.loc[(sn_remarks['possible_rca'].str.contains('long_billing_period_keywords_flag') & sn_remarks['possible_rca'].str.contains('estimate_bill_keywords_flag')), 'combined_possible_rca'] = 'estimate_bill_keywords_flag'

In [99]:
sn_remarks.loc[(sn_remarks['possible_rca'].str.contains('long_billing_period_keywords_flag') & sn_remarks['possible_rca'].str.contains('incorrect_reading_keywords_flag')), 'combined_possible_rca'] = 'incorrect_reading_keywords_flag'

In [100]:
sn_remarks.combined_possible_rca.value_counts()

estimate_bill_keywords_flag        202
meter_quality_keywords_flag        131
incorrect_reading_keywords_flag     41
Name: combined_possible_rca, dtype: int64

In [101]:
#hc.loc[hc["ID"] == '','ID'] = hc["First Name"] + hc["Last Name"]
#sn_remarks['combined_possible_rca'] = 
sn_remarks.loc[sn_remarks['combined_possible_rca'] == '','combined_possible_rca'] = sn_remarks['possible_rca']

In [101]:
sn_remarks.to_csv("C:\\Users\\ijohan\\Desktop\\ai_project\\source_data\\july\\JUNE_FURTHER_INVESTIGATION_run22July_rca_combined.csv")

# Separating flaged and further investigation bucket

In [102]:
len(sn_remarks)

18022

In [103]:
sn_remarks.flag_sum.value_counts()

0    6480
1    5874
2    4644
3     952
4      68
5       4
Name: flag_sum, dtype: int64

In [104]:
sn_remarks.flag_sum.value_counts()

0    6480
1    5874
2    4644
3     952
4      68
5       4
Name: flag_sum, dtype: int64

In [100]:
sn_remarks_flagged = sn_remarks[sn_remarks['flag_sum'] != 0]

In [101]:
sn_remarks_further = sn_remarks[sn_remarks['flag_sum'] == 0]

In [102]:
len(sn_remarks_flagged)

25376

In [103]:
len(sn_remarks_further)

15256